# Tables introduction

The Astropy [Table](http://docs.astropy.org/en/stable/table/index.html) class provides an extension of NumPy structured arrays for storing and manipulating heterogeneous tables of data. A few notable features of this package are:

- Initialize a table from a wide variety of input data structures and types.
- Modify a table by adding or removing columns, changing column names, or adding new rows of data.
- Handle tables containing missing values.
- Include table and column metadata as flexible data structures.
- Specify a description, units, and output formatting for columns.
- Perform operations like database joins, concatenation, and grouping.
- Manipulate multidimensional columns.
- Methods for reading and writing Table objects to files.
- Integration with Astropy [Units and Quantities](http://astropy.readthedocs.org/en/stable/units/index.html).

Tables vs. Pandas DataFrames
------------------------------------

The [Pandas](http://pandas.pydata.org/pandas-docs/stable/) package provides a powerful, high-performance table object via the [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame) class.  This object is similar to Astropy Table in many ways and is rather broadly used in data science. If you have some familiarity with Pandas, you might wonder why it is not used instead of an astronomy-specific solution.

There are a few reasons why Pandas is not used as a generalized table object in astronomy. These reasons include:
* Pandas does not support table "columns" that are more complicated than simple numbers. Astropy `Table` does, enabling all of the features of Astropy objects to be used inside `Table`s. 
* Pandas does not support for multidimensional table columns. This is commonly used in standard FITS data products, for instance the Chandra energy response matrix used to analyze spectral data.  
* Pandas' support for missing data relies on NaNs instead of masks. For some astronomical applications this is not desired, as it deletes potentially useful information and makes some standard astronomical analysis tasks overly complicated.

However, Pandas DataFrames have some other features that make them more natural in certain contexts, so in some sense they are complementary to Astropy Tables.  The `to_pandas` and `from_pandas` functions detailed below allow these objects to be used interchangably (as long as one of the problems mentioned above does not apply).

Documentation
------------------

For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) docs.

*****
*****


# Tutorial 

In [ ]:
from astropy.table import Table

import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt

## Creating tables

There is great deal of flexibility in the way that a table can be initially constructed:

- Read an existing table from a file or web URL
- Add columns of data one by one
- Add rows of data one by one
- From an existing data structure in memory:

  - List of data columns
  - Dict of data columns
  - List of row dicts
  - NumPy homgeneous array or structured array
  - List of row records
  
See the documentation section on [Constructing a table](http://astropy.readthedocs.org/en/stable/table/construct_table.html) for the details and plenty of examples.

In [ ]:
t = Table()
t['name'] = ['larry', 'curly', 'moe', 'shemp']
t['flux'] = [1.2, 2.2, 3.1, 4.3]

## Looking at your table

In IPython notebook, showing a table will produce a nice HTML representation of the table:

In [ ]:
t

If you did the same in a terminal session you get a different view that isn't as pretty but does give a bit more information about the table:

    >>> t
    <Table rows=4 names=('name','flux')>
    array([('source 1', 1.2), ('source 2', 2.2), ('source 3', 3.1),
           ('source 4', 4.3)], 
          dtype=[('name', 'S8'), ('flux', '<f8')])

To get a plain view which is the same in notebook and terminal use `print()`:

In [ ]:
print(t)

To get the table column names and data types using the `colnames` and `dtype` properties:

In [ ]:
t.colnames

In [ ]:
t.dtype

Astropy 1.1 and later provides a ``show_in_notebook()`` method that allows more interactive exploration of tables. It can be especially handy for large tables. 

In [ ]:
t.show_in_notebook()

## Accessing parts of the table

We can access the columns and rows as for NumPy structured arrays. Notice that the outputs are `Column`, `Row`, or `Table` objects depending on the context.

In [ ]:
t['flux']  # Flux column (notice meta attributes)

In [ ]:
t['flux'][1]  # Row 1 of flux column

In [ ]:
t[1]  # Row obj for with row 1 values

In [ ]:
t[1]['flux']  # Flux column of row 1

In [ ]:
t[1:3]  # 2nd and 3rd rows in a new table

In [ ]:
t[[1, 3]]

**One of the most powerful concepts is using boolean selection masks to filter tables**

In [ ]:
mask = t['flux'] > 3.0  # Define boolean mask for all flux values > 3
t[mask]  # Create a new table with only the "high flux" sources

## Modifying the table

Once the table exists with defined columns, there are a number of ways to modify the table in place. These are fully documented in the section [Modifying a table](http://astropy.readthedocs.org/en/stable/table/modify_table.html#modifying-a-table).

To give a few examples, add rows with the [add_row()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.add_row) method, add new columns using dict-style assignment, or assign a unit to a row:

In [ ]:
t.add_row(('joe', 10.1))  # Add a new source at the end
t['logflux'] = np.log10(t['flux'])  # Compute the log10 of the flux
t

Notice that the `logflux` column has too many output digits given the precision of the input values. We can fix this by setting the format using normal Python formatting syntax:

In [ ]:
t['flux'].format = '%.2f'
t['logflux'].format = '%.2f'
t

If you want to annotate your columns with their unit, you can add a unit to any column.  Note that this can be a string *or* an Astropy unit object. If you are familiar with using Astropy `Quantity` objects, more on that in the context of tables is described below.

In [ ]:
t['flux']. unit = 'Jy'
t

## Reading data from files into tables

You can read data using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read) method:

In [ ]:
t4 = Table.read('table_example.fits')
t4

Some formats, such as FITS and HDF5, are automatically identified by file extention while most others will require ``format`` to be explicitly provided.  `astropy` also provides a large set of ASCII file readers that will read a variety of obscure astronomy formats.  While these are actually *implemented* in the `astropy.io.ascii` sub-package, in general it is more convenient to use the "unified" I/O to read these directly as tables, like in the examples below.

In [ ]:
Table.read('example.dat', format='ascii')

In [ ]:
Table.read('example.csv', format='ascii.csv')

Note that the reader is often smart enough to determine the type of ASCII file even if you don't specify it (i.e., the `'ascii.csv'` in the above example). For example, an IPAC format file can be read by doing:

In [ ]:
Table.read('example.ipac', format='ascii')

## Writing tables to files

Similarly, tables can be written out to files in standard formats.  Many of the same formats are supported, although the exact details of which are written (and how to get them to look the way you want) can be found in the [docs for astropy.io.ascii](http://docs.astropy.org/en/stable/io/ascii/index.html).

In [ ]:
t.write('test.txt', format='ascii', overwrite=True)
!cat test.txt

Astropy also provides a special ASCII format called "ECSV" that is designed to also include metadata allowing the table to be reconstructed in Python.

In [ ]:
Table.read('test.txt' ,format='ascii')  # note that the units and formatting of the original table are lost

In [ ]:
t.write('test.ecsv', format='ascii.ecsv', overwrite=True)
Table.read('test.ecsv', format='ascii.ecsv')  # the units and formatting are as for the original table

Non-ASCII formats are available for writing too:

In [ ]:
t.write('test.fits', overwrite=True)

In [ ]:
t.write('test.vot', format='votable', overwrite=True)
!head test.vot

## Storing more complex data as table columns

Some of the most powerful usage of Astropy `Table` comes from combining two Astropy objects in a table. For example, the below illustrates a basic time-series that uses Astropy `Time` objects and a `Quantity`.

In [ ]:
from astropy.time import Time
from astropy import units as u

In [ ]:
# observations were three times over the last three hours
times = Time.now() - [2, 1, 0]*u.hour
fluxes = [21.1, 21.2, 21.25] * u.mag

ts =  Table(names=['obstime', 'flux'],
            data=[times, fluxes])
ts

In [ ]:
ts['obstime'].jd

In [ ]:
ts['flux'] + .12*u.mag

One final related note: for historical reasons, `Table` does not always behave the way you'd expect with `Quantity` objects.  As a result, the `QTable` object was developed, which is identical to `Table` with the exception of when a column has a unit, it acts like a `Quantity`.  So if you intend to use a table as a `Quantity`, you're usually better off using `QTable`:

In [ ]:
ts['flux']  # this does *not* look/act exactly like a Quantity object

In [ ]:
from astropy.table import QTable
qts = QTable(ts)
qts['flux'] # but this one does - it *is* a Quantity

## High-level table operations

So far we've only worked with one table at a time and viewed that table as a monolithic entity.  Astropy also supports high-level [Table operations](http://astropy.readthedocs.org/en/stable/table/operations.html) that manipulate multiple tables or view one table as a collection of sub-tables (groups).

 Documentation	                                                                         | Description
---------------------------------------------------------------------------------------- |-----------------------------------------
[Grouped operations](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) | Group tables and columns by keys
[Stack vertically](http://astropy.readthedocs.org/en/stable/table/operations.html#id3)   | Concatenate input tables along rows
[Stack horizontally](http://astropy.readthedocs.org/en/stable/table/operations.html#id4) | Concatenate input tables along columns
[Join](http://astropy.readthedocs.org/en/stable/table/operations.html#join)              | Database-style join of two tables

Here we'll introduce the join operation but go into more detail on the others in the exercises.

In [ ]:
from astropy.table import join

Recall our original table `t`:

In [ ]:
t

Now say that we have some additional flux values from a different reference for a different, but overlapping, sample of sources:

In [ ]:
t2 = Table()
t2['name'] = ['larry', 'moe', 'groucho']
t2['flux2'] = [1.4, 3.5, 8.6]

Now we can get a master table of flux measurements which are joined matching the values in the `name` column. This includes every row from each of the two tables, which is known as an **`outer`** join.

In [ ]:
t3 = join(t, t2, keys=['name'], join_type='outer')
print(t3)

In [ ]:
np.mean(t3['flux2'])

Alternately, we could choose to keep only rows where both tables have a valid measurement using an **`inner`** join:

In [ ]:
join(t, t2, keys=['name'], join_type='inner')

## Masked tables

As discussed above, Astropy `Table` supports masking using more than just "placeholder" values — masked columns can have both a value *and* mask information.

In [ ]:
t2 = Table([['x', 'y', 'z'], 
            [1.1, 2.2, 3.3]],
           names=['name', 'value'],
           masked=True)
t2

In [ ]:
t2['value'].mask = [False, True, False]

In [ ]:
print(t2)

In [ ]:
t2['value'].fill_value = -99
print(t2.filled())

## Interfacing with Pandas
Astropy `Table` includes  `to_pandas()` and `from_pandas()` [methods](http://docs.astropy.org/en/stable/table/pandas.html)  that facilitate conversion to/from [pandas](http://pandas.pydata.org) `DataFrame` objects.  There are a few caveats in making these conversions:
 - Tables with multi-dimensional columns cannot be converted.
 - Masked values are converted to `numpy.nan`. Numerical columns, int or float, are thus converted to ``numpy.float`` while string columns with missing values are converted to object columns with ``numpy.nan`` values to indicate missing or masked data. Therefore, you cannot always round-trip between `Table` and `DataFrame`.

In [ ]:
t_2mass = Table.read('2mass.tbl', format='ascii')

In [ ]:
import pandas as pd

pd_2mass = t_2mass.to_pandas()
pd_2mass

In [ ]:
t_pd = Table.from_pandas(pd_2mass)
t_pd.show_in_notebook()

As an example of why this might be useful, Pandas can import/export Excel files (which Astropy's io does *not* support), and this provides a way to get such data into `astropy.table`.

In [ ]:
pd_2mass.to_excel("2mass.xlsx", index=False)
excel_data = Table.from_pandas(pd.read_excel("2mass.xlsx"))
excel_data.show_in_notebook()

`DataFrame` has a built-in `plot()` method which is handy for quick visualization of its contents.

In [ ]:
pd_2mass.plot.scatter('j_m', 'h_m')

*****
*****

# Exercises


### Read the data

To start with, read in the two data files representing the master source list and observations source list.  The fields for the two tables are respectively documented in:

- [master_sources](http://cxc.harvard.edu/csc/columns/master.html)
- [obs_sources](http://cxc.harvard.edu/csc/columns/persrc.html)

In [ ]:
master_sources = Table.read('cdfs_master_sources.fits')
obs_sources = Table.read('cdfs_obs_sources.fits')

**`master_sources`**

Each distinct X-ray source identified on the sky is represented in the catalog by a single "master source" entry and one or more "source observation" entries, one for each observation in which the source has been detected. The master source entry records the best estimates of the properties of a source, based on the data extracted from the set of observations in which the source has been detected.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
msid  | Master source ID
name  | Source name in the Chandra catalog
ra  | Source RA (deg)
dec | Source Dec (deg)

**`obs_sources`**

The individual source entries record all of the properties about a detection extracted from a single observation, as well as associated file-based data products, which are observation-specific.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
obsid | Observation ID
obi | Observation interval
targname | Target name
gti_obs | Observation date
flux_aper_b | Broad band (0.5 - 7 keV) flux (erg/cm2/sec)
src_cnts_aper_b | Broad band source counts
ra_b | Source RA (deg)
dec_b | Source Dec (deg)
livetime | Observation duration (sec)
posid | Position ID
theta | Off-axis angle (arcmin)
msid | Master source ID

### Exploring the data
Do the following to explore the two tables:

- Display the data for each table in IPython notebook using the normal way of showing the value of a variable.
- Get a list of the column names for each table.  *Hint*: use `<TAB>` completion to conveniently discover all of the attributes and methods (e.g., type `master_sources.` and then hit the `<TAB>` key).
- Find the length of each table.
- Find the column datatypes for each table.

Normally you display a table in IPython notebook by entering the variable name in a cell and pressing `shift-Enter`.  In a terminal session the default method is using something like `print(my_table)`.  In both cases the `Table` object prefers to display only a screenful of data to prevent having a zillion lines of output if the table is huge. If you really want to see all of the data you can use the [Table.pprint](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.pprint) method. If you are using a Jupyter notebook interface, try the `show_in_notebook()` method.

- Display all of the rows of the `master_sources` table using its `pprint()` method.
- If you are working in a regular terminal window (not IPython notebook), try the `more()` method as well.

### Modifying tables
For our analysis we don't actually need the `obi` (observation interval) column in the `obs_sources` table.

- Remove the `obi` column from the `obs_sources` table.

The `gti_obs` column name is a bit obscure (GTI is a good time interval, FWIW).

- Rename the `gti_obs` column to `obs_date`.

It would be nice to have a count rate in addition to the source counts.

- Add a new column `src_rate_aper_b` which is the source counts divided by observation duration in sec.

Some of the sources have a negative net flux in the broad band.

### Looking at the observation source data
For each source detected in an individual observation (in the `obs_sources` table), let's look at the source flux values.

- Use the matplotlib [`hist()`]( http://matplotlib.org/api/pyplot_api.html?highlight=pyplot.hist#matplotlib.pyplot.hist) function to make a histogram of the source fluxes.  Since the fluxes vary by orders of magnitude,
  use the `numpy.log10` to put the fluxes in log space.

- Also make the same plot but using only sources within 4 arcmin of the center.  *HINT*: use a boolean mask to select values of `theta` that are less than 4.0.

### Join the master_sources and obs_sources tables

The `master_sources` and `obs_sources` tables share a common `msid` column. What we now want is to join the master RA and Dec positions and master source names with the individual observations table.

- Use the [table.join()](http://astropy.readthedocs.org/en/stable/table/operations.html#join) function to make a single table called `sources` that has the master RA, Dec, and name included for each observation source.

*HINT*: the defaults for `keys` and `join_type='inner'`  are correct in this case, so the simplest possible call to `join()` will work!

- *Intermediate*: Is the length of the new `sources` the same as `obs_sources`?  What happened?

- *Advanced*: Make a scatter plot of the RA (x-axis) and Dec (y-axis) difference between the master source position and the observation source position.  You'll need to use `coordinates`!

### Grouped properties of `sources`

Finally, we can look at the variability properties of sources in the CDFS using the [`group_by()`](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) functionality.  

This method makes a new table in which all of the sources with identical master ID are next to each other.

- Make a new table `g_sources` which is the `sources` table grouped by the `msid` key using the `group_by()` method.

The `g_sources` table is just a regular table with all of the `sources` in a particular order.  The attribute `g_sources.groups` is an object that provides access to the `msid` subgroups.  You can access the $i^{th}$ group with `g_sources.groups[i]`.

In addition, the `g_sources.groups.indices` attribute is an array with the indicies of the group boundaries.

- Using `np.diff()` find the number of repeat observations of each master sources.  *HINT*: use the indices, Luke.
- Print the 50th group and note which columns are the same for all group members and which are different.  Does this make sense?  In these few observations how many different target names were provided by observers?

#### Aggregation

The real power of grouping comes in the ability to create aggregate values for each of the groups, for instance the mean flux for each unique source.  This is done with the [`aggregate()`](http://astropy.readthedocs.org/en/stable/table/operations.html#aggregation) method, which takes a function reference as its input.  This function must take as input an array of values and return a single value.

`aggregate` returns a new table that has a length equal to the number of groups.

- Compute the mean of all columns for each unique source (i.e., each group) using `aggregate` and the `np.mean` function.  Call this table `g_sources_mean`.
- Notice that aggregation cannot form a mean for certain columns and these are dropped from the output.  Use the `join()` function to restore the `master_sources` information to `g_sources_mean`.